In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

=====> /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12



Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);


make: `/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1' is up to date.
Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Calling /Users/rob/Projects/StanSupport/cmdstan/bin/stansummary to infer across chains.

Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.99, 1.0, 0.95, 1.0) seconds, 4.0 seconds total
Sampling took (1.1, 1.1, 1.0, 1.2) seconds, 4.4 seconds total

                    Mean     MCSE   S

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE         MCSE         ESS   
         lp__  927.941804500  2.9891070874 0.033419233197 0.06475468875 2000.00000
accept_stat__    0.948571493  0.0730805694 0.000817065605 0.00089282191 2000.00000
   stepsize__    0.046459325  0.0009336965 0.000010439044 0.00010504250   79.00988
  treedepth__    5.932625000  0.4284956948 0.004790727508 0.00997107169 1846.75485
 n_leapfrog__   75.522000000 35.8753452394 0.401098553358 0.81170048102 1953.44223
  divergent__    0.000000000  0.0000000000 0.000000000000 0.00000000000        NaN
     energy__ -921.491471500  3.9119055434 0.043736433583 0.08094800048 2000.00000
            a    1.090562843  0.7197391165 0.008046927952 0.01248853286 2000.00000
           bp    0.262065123  0.0784325928 0.000876903045 0.00137570016 2000.00000
  a_society.1   -0.200181175  0.2379586211 0.002660458

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*